In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from transformers import ViTForImageClassification, ViTFeatureExtractor
from PIL import Image
import os
import torch.optim as optim
from tqdm import tqdm

# Define a custom dataset class
class ImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Set paths and labels
image_folders = [
    "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/cv-learnings/coco_ankiita/SD2Q/Train/Facebook",
    "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/cv-learnings/coco_ankiita/SD2Q/Train/Instagram",
    "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/cv-learnings/coco_ankiita/SD2Q/Train/Instagram",
    "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/cv-learnings/coco_ankiita/SD2Q/Train/Telegram",
]

image_paths = []
labels = []

# Assume folder1 has label 0, folder2 has label 1, etc.
for i, folder in enumerate(image_folders):
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        image_paths.append(img_path)
        labels.append(i)

# Define transformations and dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

dataset = ImageDataset(image_paths, labels, transform=transform)
data_loader = DataLoader(dataset, batch_size=512, shuffle=True)

# Load the ViT model and move it to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
model.to(device)

# Define the optimizer and loss function
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
num_epochs = 10
model.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in tqdm(data_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs.logits, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print epoch loss
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(data_loader):.4f}")

# Save the trained model
torch.save(model.state_dict(), "vit_classification_model.pth")

# Evaluate the model (optional)
def evaluate_model(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.logits, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total:.2f}%')

# If you have a separate validation set, use the evaluate_model function
# For now, evaluating on the same training set
evaluate_model(model, data_loader)


2024-08-13 00:51:05.756591: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-13 00:51:05.780993: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-13 00:51:05.781069: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-13 00:51:05.797705: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-13 00:51:06.753597: W tensorflow/compiler/tf

Epoch [1/10], Loss: 5.3261


100%|██████████| 6/6 [02:42<00:00, 27.15s/it]


Epoch [2/10], Loss: 1.9100


100%|██████████| 6/6 [02:46<00:00, 27.70s/it]


...

100%|██████████| 6/6 [08:02<00:00, 80.43s/it]


Epoch [9/10], Loss: 0.8541


100%|██████████| 6/6 [07:26<00:00, 74.39s/it]


Epoch [10/10], Loss: 0.7778
Accuracy: 49.79%


: 

In [1]:
# import torch
# from torch.utils.data import DataLoader, Dataset
# from torchvision import transforms
# from transformers import DeiTForImageClassification, DeiTFeatureExtractor
# from PIL import Image
# import os
# import torch.optim as optim
# from tqdm import tqdm

# # Define a custom dataset class
# class ImageDataset(Dataset):
#     def __init__(self, image_paths, labels, transform=None):
#         self.image_paths = image_paths
#         self.labels = labels
#         self.transform = transform

#     def __len__(self):
#         return len(self.image_paths)

#     def __getitem__(self, idx):
#         image = Image.open(self.image_paths[idx]).convert("RGB")
#         label = self.labels[idx]
#         if self.transform:
#             image = self.transform(image)
#         return image, label

# # Set paths and labels
# image_folders = [
#     "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/cv-learnings/coco_ankiita/SD2Q/Train/Facebook",
#     "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/cv-learnings/coco_ankiita/SD2Q/Train/Instagram",
#     "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/cv-learnings/coco_ankiita/SD2Q/Train/Instagram",
#     "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/cv-learnings/coco_ankiita/SD2Q/Train/Telegram",
# ]

# image_paths = []
# labels = []

# # Assume folder1 has label 0, folder2 has label 1, etc.
# for i, folder in enumerate(image_folders):
#     for img_name in os.listdir(folder):
#         img_path = os.path.join(folder, img_name)
#         image_paths.append(img_path)
#         labels.append(i)

# Define transformations and dataset
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
# ])

# dataset = ImageDataset(image_paths, labels, transform=transform)
# data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# # Load the DeiT-Tiny model and move it to the GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# from transformers import DeiTForImageClassification

# # Clear cache and retry
# DeiTForImageClassification.from_pretrained('facebook/deit-tiny-patch16-224', force_download=True)

# model.classifier = torch.nn.Linear(model.config.hidden_size, len(image_folders))  # Adjusting output layer to match number of classes
# model.to(device)

# # Define the optimizer and loss function
# optimizer = optim.AdamW(model.parameters(), lr=5e-4)
# criterion = torch.nn.CrossEntropyLoss()

# # Training loop
# num_epochs = 10
# model.train()

# for epoch in range(num_epochs):
#     running_loss = 0.0
#     for images, labels in tqdm(data_loader):
#         images, labels = images.to(device), labels.to(device)

#         optimizer.zero_grad()

#         # Forward pass
#         outputs = model(images)
#         loss = criterion(outputs.logits, labels)

#         # Backward pass and optimization
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#     # Print epoch loss
#     print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(data_loader):.4f}")

# # Save the trained model
# torch.save(model.state_dict(), "deit_tiny_classification_model.pth")

# # Evaluate the model (optional)
# def evaluate_model(model, data_loader):
#     model.eval()
#     correct = 0
#     total = 0
#     with torch.no_grad():
#         for images, labels in data_loader:
#             images, labels = images.to(device), labels.to(device)
#             outputs = model(images)
#             _, predicted = torch.max(outputs.logits, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()

#     print(f'Accuracy: {100 * correct / total:.2f}%')

# # If you have a separate validation set, use the evaluate_model function
# # For now, evaluating on the same training set
# evaluate_model(model, data_loader)


In [ ]:
# Define your custom model
model = create_crossvit_model(input_shape=(224, 224, 7), num_classes=4)

2024-08-13 00:21:50.224955: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79090 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:01:00.0, compute capability: 8.0
2024-08-13 00:21:50.226682: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 79090 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:41:00.0, compute capability: 8.0
2024-08-13 00:21:50.228045: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 79090 MB memory:  -> device: 2, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:81:00.0, compute capability: 8.0
2024-08-13 00:21:50.229414: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 79090 MB memory:  -> device: 3, name: NVIDIA A100-SXM4-80GB, pci bu

UnknownError: {{function_node __wrapped__Log_device_/job:localhost/replica:0/task:0/device:GPU:0}} JIT compilation failed. [Op:Log] name: 